# MEC_51053_EP — Numerical methods in (solid) mechanics — Martin Genet

# L9 — Partial differential equations (Elastodynamics initial/boundary value problem) — The finite element method + Integration schemes 

# E9.2 — Dynamics — Matthias Rambausek & Martin Genet

As usual, let us start with *some* imports…

In [ ]:
# For better printing within jupyter cells (otherwise only the last variable is printed)
import IPython
IPython.core.interactiveshell.InteractiveShell.ast_node_interactivity = "all"

# Standard python libraries
import math
import os
import time

# Computing libraries
import numpy

# Plotting libraries
import matplotlib.pyplot as plt

# Meshing libraries
import gmsh
import meshio

# FEniCS
import dolfin # dolfin if the main interface to FEniCS
dolfin.parameters["form_compiler"]["cpp_optimize"] = True
dolfin.parameters["form_compiler"]["optimize"] = True

# VTK and visualization
import itkwidgets
import myVTKPythonLibrary as myvtk

# LIB552 python library
import LIB552 as lib

## Problem

In this exercise we compute the dynamic response of a thin disc (radius $R$, thickness $Z$) made of an isotropic linear elastic material (mass density $\rho$, Young modulus $E$, Poisson ratio $\nu$) to an applied force.
In principle one would use a plate model for such a problem, but here we will model it in 3D.

The force is applied on a region of the top surface of the disc (center $\underline{x_0}$, radius $r_0$).
It corresponds to a vertical force that increases linearly from 0 at time 0 to $f_0$ at time $t_0$, and then stays constant.
However, the analysis is continued until time $T > t_0$.

### Parameters

We set some numerical values for the physical parameters.

#### Geometrical

In [ ]:
R = 1.
Z = 0.1

#### Material

**Q1.
Express the shear wave velocity ($c_T$) in a linear isotropic continuum as a function of mass density $\rho$ and shear modulus $\mu$.**

**Q2.
Complete and execute the following code.**

In [ ]:
rho   = 1.
E     = 1.
nu    = 0.3
lmbda = ### YOUR CODE HERE ###
mu    = ### YOUR CODE HERE ###
cT    = ### YOUR CODE HERE ###
print ("cT:", cT)

#### Structural

**Q3. Express the time $\tau_0$ it takes to a shear wave to travel through the disc radius $R$.**

**Q4.
Complete and execute the following code.**

In [ ]:
tau0 = ### YOUR CODE HERE ###
print ("tau0:", tau0)

#### Loading

We first consider an extremely slow (almost quasi-static) loading, by setting $t_0$ to, e.g., a hundred times $\tau_0$.

In [ ]:
x0   = numpy.array([0., 0.])
r0   = 0.2
f0   = 0.1
t0   = 100*tau0
print ("t0:", t0)

#### Analysis

The total duration $T$ can be set to, e.g., two times $t_0$.

In [ ]:
T = 2*t0
print ("T:", T)

## Finite element resolution

### Parameters

We define the computation parameters.

#### Spatial discretization

In [ ]:
# Number of elements in the radius of the disc
N_R = 10

# Number of elements in the thickness of the disc
N_Z = 5

#### Temporal discretization

Since the first computation is almost quasi-static, we will start with a rather large time step (e.g., ten times $\tau_0$), and use an implicit time scheme (Newmark).
Later on we will use an explicit time scheme (central differences).

Note that even though the central differences scheme corresponds to specific values of parameters of the Newmark scheme, we will use two separate implementations, in order to exploit the explicit nature of the central differences scheme to make it (much) faster.

**Q5. What values of parameters $\gamma$ & $\beta$ make the Newmark scheme implicit, with good energy preservation properties?**

**Q6. For the central differences scheme, what is the approximated stability limit for the time step?**

**Q7.
Complete and execute the following code.**

In [ ]:
# Integration scheme
scheme = "newmark"; gamma = ### YOUR CODE HERE ###; beta = ### YOUR CODE HERE ###
# scheme = "central_differences"

# Approximated stability time step for explicit schemes
dt_max_for_cdc = ### YOUR CODE HERE ###
print("dt_max_for_cdc:", dt_max_for_cdc)

# Time step
dt = 10*tau0
print("dt:", dt)

# Number of time steps (note that it must be an integer)
n_t = int(T/dt)
print("n_t:", n_t)

#### Visualization

Especially for explicit time schemes that require small —and thus many— time steps, one might not want to save all time steps, which would take time at each step and make the visualization heavy.
Here we define the maximum number of visualization step, and compute the visualization time step from it.

In [ ]:
# Maximum number of visualization steps
n_v_max = 100
print("n_v_max:", n_v_max)

# Actual number of visualization steps (note that it cannot be larger than the number of computation time steps)
n_v = min(n_v_max, n_t)
print("n_v:", n_v)

# Visualization time step (note that it must be a multiple of the computation time step)
dt_v = int(n_t/n_v) * dt
print("dt_v:", dt_v)

### Mesh

We build the mesh, for instance using constructive geometry.
In order to have a fine control on the element size (notably, different refinements in the plane in the thickness), we first generate a disc, and then extrude it.
Note that the disc must be between $-Z/2$ and $+Z/2$.

**Q8.
Complete and execute the following code.**

Hints:
* `factory.addDisk(xc, yc, zc, rx, ry)` creates a (2D) ellipsoid centered in [xc,yc,zc], aligned with the (x,y) plane, with radii [rx,ry], and returns its tag.
* `factory.extrude(dimTags=[(2,s1)], dx, dy, dz, numElements=[n])[1][1]` extrudes the (2D) surface tagged `s1` by the vector [dx, dy, dz], creating n layers of elements, and returns the tag of the generated (3D) volume.

In [ ]:
## Initialization
gmsh.initialize()
gmsh.clear()

## Geometry
factory = gmsh.model.occ
disk_surf_tag = ### YOUR CODE HERE ###
disk_volu_tag = ### YOUR CODE HERE ###

# Synchronization, cf., e.g., https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/python/t16.py
factory.synchronize()

# In order to only save nodes and elements of the final mesh
# (i.e., not the construction nodes and elements—remember that 
# unstructured meshers will first mesh the curves, then the
# surfaces and the volumes, cf. L5.2), we declare it as a
# "physical" entity.
disk_phys_tag = gmsh.model.addPhysicalGroup(dim=3, tags=[disk_volu_tag])

## Mesh
mesh_gmsh = gmsh.model.mesh

# Characteristic size, cf., e.g., https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/python/t16.py
mesh_gmsh.setSize(gmsh.model.getEntities(0), R/N_R)

# Mesh generation
mesh_gmsh.generate(dim=3)

# In order to visualize the mesh and perform finite element computation using
# FEniCS, we need to convert the mesh from the GMSH format to the VTK & FEniCS
# formats. Since there is no direct converter between these formats, we do
# that here by writing the mesh to the disc in VTK format using GMSH, which
# we can then read in various formats later on.
gmsh.write("E9.2-Dynamics-mesh.vtk")

# Finalization
gmsh.finalize()

Let us visualize the mesh, using [itkwidgets](https://github.com/InsightSoftwareConsortium/itkwidgets).

In [ ]:
mesh_vtk = myvtk.readUGrid("E9.2-Dynamics-mesh.vtk")
itkwidgets.view(geometries=mesh_vtk)

Let us now convert the mesh into the FEniCS format, using [meshio](https://github.com/nschloe/meshio).

In [ ]:
mesh_meshio = meshio.read("E9.2-Dynamics-mesh.vtk")
meshio.write("E9.2-Dynamics-mesh.xdmf", mesh_meshio)

mesh = dolfin.Mesh()
dolfin.XDMFFile("E9.2-Dynamics-mesh.xdmf").read(mesh)

We define the subdomains needed to impose boundary conditions and loading.

In [ ]:
# AutoSubDomain is a convenient way to define a SubDomain object
# that can be used to mark certain parts of the mesh
ext_sd = dolfin.AutoSubDomain(
    lambda x, on_boundary: on_boundary and (numpy.linalg.norm(x[0:2]) > 0.99*R))
f_sd = dolfin.AutoSubDomain(
    lambda x, on_boundary: on_boundary and dolfin.near(x[2], +Z/2) and (numpy.linalg.norm(x[0:2]-x0) <= r0))

# Create a 'MeshFunction' that lets us mark certain parts of the boundary
boundaries_mf = dolfin.MeshFunction("size_t", mesh, mesh.topology().dim()-1)

# Initialize to zero
boundaries_mf.set_all(0)

# Mark the different boundaries
ext_id = 1
ext_sd.mark(boundaries_mf, ext_id)
f_id = 2
f_sd.mark(boundaries_mf, f_id)

# We can visualize the boundaries
dolfin.File("E9.2-Dynamics-boundaries_.pvd").write(boundaries_mf)
boundaries_vtk = myvtk.readUGrid("E9.2-Dynamics-boundaries_000000.vtu")
itkwidgets.view(geometries=boundaries_vtk)

### System matrix & vector

We define the finite element approximation.

In [ ]:
# Finite element
fe = dolfin.VectorElement(
    family="Lagrange",
    cell=mesh.ufl_cell(), # triangle in 2D, tetrahedron in 3D
    degree=1)

# Function space
fs = dolfin.FunctionSpace(mesh, fe)

# Current (unknown) & previous (known) displacement, velocity & acceleration fields
u, u_old = dolfin.Function(fs, name="u"), dolfin.Function(fs, name="u_old")
v, v_old = dolfin.Function(fs, name="v"), dolfin.Function(fs, name="v_old")
w, w_old = dolfin.Function(fs, name="w"), dolfin.Function(fs, name="w_old")

We define the test & trial function for defining variational forms.
These are the objects that formally get replaced by the linear combinations of shape functions in order to define the associated vector/matrix.

In [ ]:
# The test function is the U* of the slides
u_test = dolfin.TestFunction(fs)

# The trial function is the unknown of the variational problem, it is the U of the slides
u_tria = dolfin.TrialFunction(fs)

We now build the system mass matrix.

**Q9.
Complete and execute the following code**

Hints:
 * `dolfin.inner(f, g)` returns the inner (i.e., scalar) product between the two tensors `f` & `g`.
(If `f` & `g` are zero-order tensors, i.e., scalars, it is a simple product;
if they are first-order tensors, i.e., vectors, it is a dot product;
if they are second-order tensors, i.e., matrices, it is a double dot product; …)
 * `f * dolfin.dx(domain=mesh)` is the variational form corresponding to the integral of the scalar function `f` over the domain `mesh`.
 * `dolfin.assemble(form)` will assemble the variational form `form` and return the associated object.
(If `form` has arity 0, i.e., if it is a scalar, independent from any test or trial function, it will return its integral, i.e., a scalar;
if it has arity 1, i.e., if it is a linear form of a test function, but independent from any trial function, it will return its integral as a finite element vector;
if it has arity 2, i.e., if it is a bilinear form of a test function and a trial function, it will return its integral as a finite element matrix.)

In [ ]:
# We turn rho into a dolfin.Constant,
# so that if we change its numerical value,
# FEniCS will not regenerate and recompile
# the variational forms and associated
# assembly procedures.
rho = dolfin.Constant(rho)

# Variational form defining the mass matrix
M_form = ### YOUR CODE HERE ###

# Mass matrix
M = ### YOUR CODE HERE ###

For explicit schemes, it is convenient/efficient to build a diagonal approximation of the mass matrix, a.k.a. the lumped mass matrix.
Since it is diagonal, we only store the diagonal itself within a vector.

In [ ]:
# Lumped mass matrix, cf. https://comet-fenics.readthedocs.io/en/latest/demo/tips_and_tricks/mass_lumping.html
M_lumped_form = rho * dolfin.inner(u_tria, u_test) * dolfin.dx(domain=mesh, scheme="vertex", metadata={"degree":1, "representation":"quadrature"})
M_lumped_mat = dolfin.assemble(M_lumped_form)
M_lumped = u.vector().copy()
M_lumped_mat.get_diagonal(M_lumped)

We now build the system stiffness matrix & force vector.
In order to impose boundary conditions through symmetric substitution, we need to assemble both the stiffness matrix and force vector together.
We will assemble the full force vector (i.e., at time $t_0$), and then make a copy that we will update at each time step.

**Q10.
Complete and execute the following code**

Hints:
 * `dolfin.grad(f)` returns the gradient of the tensor `f`.
(If `f` is a zero-order tensor, i.e., a scalar, it is a first-order tensor, i.e., a vector;
if it is a first-order tensor, i.e., a vector, it is a second-order tensor, i.e., a matrix; …)
 * `dolfin.sym(f)` returns the symmetric part of the second order tensor `f`, i.e, the second order tensor (f + f.T)/2.
 * `dolfin.tr(f)` returns the trace of the second order tensor `f`.
 * `dolfin.Identity(d)` returns the identity second order tensor in dimension `d`.
 * `dolfin.inner(f, g)` returns the inner (i.e., scalar) product between the two tensors `f` & `g`.
(If `f` & `g` are zero-order tensors, i.e., scalars, it is a simple product;
if they are first-order tensors, i.e., vectors, it is a dot product;
if they are second-order tensors, i.e., matrices, it is a double dot product; …)
 * `f * dolfin.dx(domain=mesh)` is the variational form corresponding to the integral of the scalar function `f` over the domain `mesh`.
 * `dolfin.assemble_system(A_form, b_form, bcs)` will assemble the bilinear form `A_form` & linear form `b_form` and return the associated matrix & vector, while taking into account the list of boundary conditions `bcs` through symmetric substitution.

In [ ]:
# We turn lambda & mu into dolfin.Constants.
lmbda = dolfin.Constant(lmbda)
mu    = dolfin.Constant(mu)

# Stress (associated to the trial function)
eps_tria = ### YOUR CODE HERE ###
sig_tria = ### YOUR CODE HERE ###

# Strain (associated to the test function)
eps_test = ### YOUR CODE HERE ###

# Variational form defining the stiffness matrix
K_form = ### YOUR CODE HERE ###

# Variational form defining the (full) force vector
f = dolfin.Constant([0,0,-f0])
F_form = dolfin.inner(f, u_test) * dolfin.ds(domain=mesh, subdomain_data=boundaries_mf, subdomain_id=f_id)

# Boundary conditions
bcs = [dolfin.DirichletBC(fs, dolfin.Constant((0.0, 0.0, 0.0)), boundaries_mf, ext_id)]

# Stiffness matrix & (full) Force vector
K, F0 = ### YOUR CODE HERE ###

# Current force vector (which will be updated at each time step)
F = F0.copy()
F.zero()

### Resolution algorithms

In order to separate the different parts of the resolution procedure, we define objects containing specific algorithms.
This is standard object-oriented programming, allowing for (supposedly) better code readability, maintainability, reusability, etc.
We start with the time integrator, which relies on a generic time_scheme object to advance time, which we will specify later.

#### Time integrator

In [ ]:
class TimeIntegrator():
    """
    Dynamic time integrator.

    Attributes:
        u     (dolfin.Function): Displacement at the end of the current time step
        u_old (dolfin.Function): Displacement at the end of the previous time step
        v     (dolfin.Function): Velocity at the end of the current time step
        v_old (dolfin.Function): Velocity at the end of the previous time step
        w     (dolfin.Function): Acceleration at the end of the current time step
        w_old (dolfin.Function): Acceleration at the end of the previous time step
        
        U     (dolfin.Vector): Displacement at the end of the current time step
        U_old (dolfin.Vector): Displacement at the end of the previous time step
        V     (dolfin.Vector): Velocity at the end of the current time step
        V_old (dolfin.Vector): Velocity at the end of the previous time step
        W     (dolfin.Vector): Acceleration at the end of the current time step
        W_old (dolfin.Vector): Acceleration at the end of the previous time step

        M  (dolfin.Matrix): Mass matrix
        K  (dolfin.Matrix): Stiffness matrix
        F  (dolfin.Vector): Force vector at the end of the current time step

        time_scheme (TimeScheme): Time scheme called at each time step

        dt (float): Time step
        n_t  (int): Number of time steps
        
        F0          (dolfin.Vector): Force vector at time t0
        t0                  (float): Characteristic time of the loading
        maintain_F0_after_t0 (bool): Option to maintain (default) or not the force after the characteristic time

        dt_v (float): Visualisation time step

        compute_energies (bool): Option to compute (default) or not the energies at each time step
    """

    # Initialization function
    # Note that `self` is the mandatory first keyword argument of each class function (a.k.a., method),
    #  it represents the object itself (once it is created).
    def __init__(self,
            u, u_old,
            v, v_old,
            w, w_old,
            M, K, F,
            time_scheme,
            parameters):

        self.u, self.u_old = u, u_old
        self.v, self.v_old = v, v_old
        self.w, self.w_old = w, w_old
        
        self.U, self.U_old = self.u.vector(), self.u_old.vector()
        self.V, self.V_old = self.v.vector(), self.v_old.vector()
        self.W, self.W_old = self.w.vector(), self.w_old.vector()

        self.M, self.K, self.F = M, K, F
        self.tmp = self.u.vector().copy() # sometimes we need temporary objects for linear algebra operations

        self.time_scheme = time_scheme

        self.dt  = parameters["dt"]
        self.n_t = parameters["n_t"]
        
        self.F0 = parameters["F0"]
        self.t0 = parameters["t0"]
        self.maintain_F0_after_t0 = parameters.get("maintain_F0_after_t0", "True")

        self.dt_v = parameters["dt_v"]

        self.compute_energies = parameters.get("compute_energies", "True")


    # Integration function
    def integrate(self):
        
        # Initialization
        self.U.zero(), self.U_old.zero()
        self.V.zero(), self.V_old.zero()
        self.W.zero(), self.W_old.zero()
        if (self.compute_energies): self.energies = numpy.zeros((self.n_t+1,5))
        t = 0.
        k_visu = 0

        # Visualization
        os.system("rm -rf E9.2-Dynamics")
        os.system("mkdir E9.2-Dynamics")
        self.pvd_file = dolfin.File("E9.2-Dynamics/u.pvd")
        self.pvd_file.write(self.u, t); k_visu += 1

        # Loop over time steps
        for k_t in range(self.n_t):
            tic = time.time()

            # Time step number
            print("k_t =", k_t)
            
            # Time at the end of the step
            t = (k_t+1) * self.dt
            print(" |  "+"t =", t)

            # Update fields at the beginning of the step
            self.U_old[:] = self.U
            self.V_old[:] = self.V
            self.W_old[:] = self.W

            # Update load at the end of the step
            self.F.zero()
            if (t <= self.t0):
                self.F.axpy(t/self.t0, self.F0)
            elif (self.maintain_F0_after_t0):
                self.F.axpy(       1., self.F0)

            # Update fields at the end of the step
            self.time_scheme.update()
            
            # Check if solution is sane
            if not numpy.isfinite(self.U.norm("l2")):
                print(" |  "+"Warning! Solution diverged. Aborting.")
                break

            # Compute energies
            if (self.compute_energies):
                self.energies[k_t+1,0] = t

                self.K.mult(self.U, self.tmp)
                self.energies[k_t+1,1] = self.U.inner(self.tmp)/2

                self.M.mult(self.V, self.tmp)
                self.energies[k_t+1,2] = self.V.inner(self.tmp)/2

                self.energies[k_t+1,3] = self.energies[k_t+1,1] + self.energies[k_t+1,2]

                self.tmp[:] = self.U
                self.tmp.axpy(-1., self.U_old)
                self.energies[k_t+1,4] = self.energies[k_t,4] + self.F.inner(self.tmp)

            # Visualization
            if math.isclose(t, k_visu * self.dt_v):
                print(" |  "+"k_visu =", k_visu)
                self.pvd_file.write(self.u, t); k_visu += 1
                
            toc = time.time()
            print(" |  "+"runtime =", toc-tic, "s")

#### Time scheme

Now we specify various time schemes, namely the central differences (explicit) scheme and the Newmark scheme.

Note that even though the central differences scheme corresponds to specific values of parameters of the Newmark scheme, we will use two separate implementations, in order to exploit the explicit nature of the central differences scheme to make it (much) faster.

**Q11.
Complete and execute the following code.**

Hints:
* `y.axpy(a, x)` corresponds to `y = y + a * x` where `x` and `y` are dolfin vectors.
* `dolfin.LUSolver(A)` initializes a (direct) linear solver with the dolfin matrix `A`. 
* `linear_solver.solve(X, B)` will solve the linear system `A * X = B` where `A` is a dolfin matrix provided when initializing the solver & `B` is a dolfin vector, and store the solution into the dolfin vector `X`.

In [ ]:
# Parent class for integration schemes.
# Here it is empty, but it could contain variables and functions that are useful for all schemes,
#  so that we do not have to copy-paste the code inside each child class.
class TimeScheme:
    pass

class CentralDifferencesTimeScheme(TimeScheme): # This is how you stipulate that CentralDifferencesTimeScheme
                                                # derives from (a.k.a., is a child class of) TimeScheme
    """
    Central differences time scheme.

    Attributes:
        U     (dolfin.Vector): Displacement at the end of the current time step
        U_old (dolfin.Vector): Displacement at the end of the previous time step
        V     (dolfin.Vector): Velocity at the end of the current time step
        V_old (dolfin.Vector): Velocity at the end of the previous time step
        W     (dolfin.Vector): Acceleration at the end of the current time step
        W_old (dolfin.Vector): Acceleration at the end of the previous time step
        
        M (dolfin.Vector): Diagonal vector of the lumped mass matrix
        K (dolfin.Matrix): Stiffness matrix
        F (dolfin.Vector): Force vector at the end of the current time step

        dt (float): Time step
    """
    
    # Initialization function
    # In this function we prepare everything such that the actual integration function is concise and fast.
    # It takes as an argument the various parameters, physical and numerical, of the problem.
    # Note that `self` is the mandatory first keyword argument of each class function (a.k.a., method),
    #  it represents the object itself (once it is created).
    def __init__(self,
            U, U_old,
            V, V_old,
            W, W_old,
            M, K, F,
            dt):
        self.U, self.U_old = U, U_old
        self.V, self.V_old = V, V_old
        self.W, self.W_old = W, W_old
        self.M, self.K, self.F = M, K, F
        self.dt = dt

    def update(self):
        # Update U with V_old
        ### YOUR CODE HERE ###
        # Update U with W_old
        ### YOUR CODE HERE ###
        # Update V with W_old
        ### YOUR CODE HERE ###
        # Update W
        self.K.mult(-self.U, self.W)
        self.W.axpy(1., self.F)
        self.W.vec().pointwiseDivide(self.W.vec(), self.M.vec()) # Thanks to Jérémy Bleyer!
        # Update V with W
        ### YOUR CODE HERE ###
        
        # Note that we could have used the following code,
        #  which is somewhat clearer but much slower.
#         self.U[:] +=  self.dt       * self.V_old
#         self.U[:] += (self.dt**2/2) * self.W_old
#         self.V[:] += (self.dt   /2) * self.W_old
#         self.W[:]  = self.F - self.K*self.U
#         self.W[:] /= self.M
#         self.V[:] += (self.dt   /2) * self.W

class NewmarkTimeScheme(TimeScheme):
    """
    Newmark time scheme.

    Attributes:
        U     (dolfin.Vector): Displacement at the end of the current time step
        U_old (dolfin.Vector): Displacement at the end of the previous time step
        V     (dolfin.Vector): Velocity at the end of the current time step
        V_old (dolfin.Vector): Velocity at the end of the previous time step
        W     (dolfin.Vector): Acceleration at the end of the current time step
        W_old (dolfin.Vector): Acceleration at the end of the previous time step
        
        M (dolfin.Matrix): Mass matrix
        K (dolfin.Matrix): Stiffness matrix
        F (dolfin.Vector): Force vector at the end of the current time step

        dt (float): Time step

        gamma (float): Newmark parameter
        beta  (float): Newmark parameter

        linear_solver (dolfin.LinearSolver): Linear solver
    """

    def __init__(self,
            U, U_old,
            V, V_old,
            W, W_old,
            M, K, F,
            dt,
            gamma,
            beta):
        self.U, self.U_old = U, U_old
        self.V, self.V_old = V, V_old
        self.W, self.W_old = W, W_old
        self.M, self.K, self.F = M, K, F
        self.dt    = dt
        self.gamma = gamma
        self.beta  = beta
        
        self.linear_solver = dolfin.LUSolver(### YOUR CODE HERE ###)

    def update(self):
        # Update U with V_old
        ### YOUR CODE HERE ###
        # Update U with W_old
        ### YOUR CODE HERE ###
        # Update V with W_old
        ### YOUR CODE HERE ###
        # Update W
        ### YOUR CODE HERE ###
        # Update U with W
        ### YOUR CODE HERE ###
        # Update V with W
        ### YOUR CODE HERE ###

        # Again, we could have used the following code,
        #  which is somewhat clearer but much slower.
#         self.U[:] +=  self.dt                            * self.V_old
#         self.U[:] += (self.dt**2/2) * (1 - 2*self.beta ) * self.W_old
#         self.V[:] +=  self.dt       * (1 -   self.gamma) * self.W_old
#         self.linear_solver.solve(self.W, self.F - self.K*self.U)
#         self.U[:] += (self.dt**2/2) *      2*self.beta   * self.W
#         self.V[:] +=  self.dt       *        self.gamma  * self.W

### Resolution

Now we actually solve the problem.

In [ ]:
# Let us recall the time parameters
print ("tau0:", tau0)

t0 = 100*tau0
print ("t0:", t0)

T = 2*t0
print ("T:", T)

scheme = "newmark"; gamma = 1/2; beta = 1/4
# scheme = "central_differences"

print("dt_max_for_cdc:", dt_max_for_cdc)

dt = 10*tau0
print("dt:", dt)

n_t = int(T/dt)
print("n_t:", n_t)

n_v_max = 100
n_v = min(n_v_max, n_t)
dt_v = int(n_t/n_v) * dt
print("n_v:", n_v)

In [ ]:
# Time scheme
if (scheme == "central_differences"):
    time_scheme = CentralDifferencesTimeScheme(
        U=u.vector(), U_old=u_old.vector(),
        V=v.vector(), V_old=v_old.vector(),
        W=w.vector(), W_old=w_old.vector(),
        M=M_lumped, K=K, F=F,
        dt=dt)
elif (scheme == "newmark"):
    time_scheme = NewmarkTimeScheme(
        U=u.vector(), U_old=u_old.vector(),
        V=v.vector(), V_old=v_old.vector(),
        W=w.vector(), W_old=w_old.vector(),
        M=M, K=K, F=F,
        dt=dt,
        gamma=gamma, beta=beta)

# Time integrator
time_integrator = TimeIntegrator(
    u=u, u_old=u_old,
    v=v, v_old=v_old,
    w=w, w_old=w_old,
    M=M, K=K, F=F,
    time_scheme=time_scheme,
    parameters={"dt":dt,
                "n_t":n_t,
                "F0":F0,
                "t0":t0,
                "dt_v":dt_v})

# Integration
tic = time.time()
time_integrator.integrate()
toc = time.time()
print("runtime:", toc-tic)

### Visualization

You can visualize the solution in the notebook, or using [ParaView](https://www.paraview.org).

In [ ]:
viewer = lib.DisplacementViewer(pvd_folder="E9.2-Dynamics", pvd_filename="u.pvd")
viewer.show()

We also plot various energies over time.

In [ ]:
fig = plt.figure(figsize=(12,8));
ax = fig.add_subplot(221);
ax.plot(time_integrator.energies[:,0], time_integrator.energies[:,1]);
ax.set_xlabel("time");
ax.set_ylabel("elastic energy");
ax = fig.add_subplot(222);
ax.plot(time_integrator.energies[:,0], time_integrator.energies[:,2]);
ax.set_xlabel("time");
ax.set_ylabel("kinetic energy");
ax = fig.add_subplot(223);
ax.plot(time_integrator.energies[:,0], time_integrator.energies[:,3]);
ax.set_xlabel("time");
ax.set_ylabel("internal energy");
ax = fig.add_subplot(224);
ax.plot(time_integrator.energies[:,0], time_integrator.energies[:,4]);
ax.set_xlabel("time");
ax.set_ylabel("external energy");
fig.tight_layout();

### Analysis

**Q12.
Do the energy plots make sense to you?
Why?**

**Q13.
What do you notice regarding the resolution time of the first time step compared to the next ones?
(Feel free to increase the mesh size to make this difference more apparent.)
Why is that?**

**Q14.
Use the central differences scheme instead of Newmark.
What happens?
Why?**

**Q15. Now reduce the time step to a stable value.
How many time steps do you need to compute the full solution?
What do you notice in terms of the obtained solution, in displacement & energy?**

**Q16.
What do you notice regarding the resolution time of a single time step with central differences versus Newmark?
(Feel free to increase the mesh size to make this difference more apparent.
Also, do not hesitate to comment out the lines where energies are computed, which is not negligible.)
Why is that?**

**Q17.
Now make the loading time $t_0$ smaller (e.g., 10x), modify the code such that the applied force is null after $t_0$, and try with both schemes.
What do you notice?**

**Q18.
Now make the loading time $t_0$ even smaller (e.g., 10x or even 100x), and try with both schemes.
What do you notice?**

### Bonus

**Q19.
Move the position of the applied force.
What do you notice?**

**Q20. What values of parameters $\gamma$ & $\beta$ make the Newmark scheme correspond to the central differences explicit scheme?
Now compare the computing time of the central differences algorithm and the Newmark algorithm with these parameters.
What do you notice?
Why is that?**